# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [31]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import time

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


In [11]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577 entries, 0 to 576
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     577 non-null    int64  
 1   City        577 non-null    object 
 2   Lat         577 non-null    float64
 3   Lng         577 non-null    float64
 4   Max Temp    577 non-null    float64
 5   Humidity    577 non-null    int64  
 6   Cloudiness  577 non-null    int64  
 7   Wind Speed  577 non-null    float64
 8   Country     574 non-null    object 
 9   Date        577 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 45.2+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [22]:
scale_factor = 1
city_data_df['point_size'] = city_data_df['Humidity'] * scale_factor

# Configure the map plot
city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    tiles='OSM',
    size='point_size',
    frame_width=800,
    frame_height=600
)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (point_size)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
mask = city_data_df['Max Temp'] >= 20
df = city_data_df.loc[mask].reset_index(drop=True)

# Display sample data
df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,point_size
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228,35
1,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232,76
2,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236,68
3,10,colorado,-22.8375,-51.9731,20.36,86,96,4.69,BR,1666107965,86
4,12,trairi,-3.2778,-39.2689,31.07,51,34,8.09,BR,1666108038,51


### Step 3: Create a new DataFrame called `hotel_df`.

In [29]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = '' 

# Display the updated DataFrame
print(hotel_df.head())

# Display sample data
hotel_df.info()

          City Country      Lat       Lng  Humidity Hotel Name
0         faya      SA  18.3851   42.4509        35           
1      farsund      NO  58.0948    6.8047       100           
2  new norfolk      AU -42.7826  147.0587        58           
3    jamestown      US  42.0970  -79.2353        77           
4      lanzhou      CN  36.0564  103.7922        48           
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577 entries, 0 to 576
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        577 non-null    object 
 1   Country     574 non-null    object 
 2   Lat         577 non-null    float64
 3   Lng         577 non-null    float64
 4   Humidity    577 non-null    int64  
 5   Hotel Name  577 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 27.2+ KB


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [42]:
#xpert assistant used on this sale
# Set parameters to search for a hotel within a radius of 10,000 meters
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

base_url = "https://api.geoapify.com/v2/places?"

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    hotel_url = f"{base_url}categories=accommodation.hotel&apiKey={geoapify_key}"
    
    # Make an API request using the params dictionary
    response = requests.get(hotel_url, params=params)

    print(response)
    
    # Convert the API response to JSON format
    data = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_name = data["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    #sleep for 1 second
    time.sleep(1)
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data from hotel_df
hotel_df

Starting hotel search
<Response [401]>
faya - nearest hotel: No hotel found


KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)